<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-библиотек-и-предобработка-данных" data-toc-modified-id="Загрузка-библиотек-и-предобработка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка библиотек и предобработка данных</a></span></li><li><span><a href="#Тематический-анализ-и-частотный-анализ-слов" data-toc-modified-id="Тематический-анализ-и-частотный-анализ-слов-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Тематический анализ и частотный анализ слов</a></span></li><li><span><a href="#Наиболее-проблемные-товары" data-toc-modified-id="Наиболее-проблемные-товары-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Наиболее проблемные товары</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Предложения-и-гипотезы-по-улучшению-опыта-пользователей:" data-toc-modified-id="Предложения-и-гипотезы-по-улучшению-опыта-пользователей:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Предложения и гипотезы по улучшению опыта пользователей:</a></span></li></ul></div>

# Самокат. Мороженое

**Описание**  
Самокат позволяет пользователю поставить оценку и оставить отзыв на товар.    
Мы хотим использовать эти отзывы и оценки для улучшения наших товаров и процессов.

**Дано**  
Выгрузка отзывов за последний месяц на мороженное "Самокат"   
    ``name_samokat`` - наименование товара  
    ``comment`` - отзыв на товар  
    ``rating`` - оценка заказа  

**Ожидаемый результат**  
Список гипотез "как можно улучшить опыт покупателей мороженного Самокат".  
Для гипотез, придумать как её можно проверить, описать каких дополнительных данных и исследований не хватает для её валидации?

## Загрузка библиотек и предобработка данных

In [1]:
import spacy
import gensim
import nltk
from gensim.utils import simple_preprocess
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from collections import Counter
import pandas as pd

По ходу анализа было принято решение частицы "не" объединить со следующим словом

In [2]:
data = pd.read_excel("icecream_Samokat_1.xls")

In [3]:
data

,name_samokat,comment,rating
0,"Мороженое Самокат вафельный стаканчик, пломбир...",Самое вкусное мороженое !,5
1,"Мороженое Самокат вафельный стаканчик, шоколад...",Очень вкусное ! Настоящее,5
2,"Мороженое Самокат пломбир классический, 15%, 2...",Вкуснейшее мороженое,5
3,"Мороженое Самокат с йогуртом, с клубникой, 10%...","Какая то гадость купил детям выкинули, прогорк...",1
4,"Мороженое Самокат пломбир, шоколадный, 15%, в ...",Качественный продукт,4
...,...,...,...
3080,"Мороженое Самокат вафельный стаканчик, пломбир...","Самое вкусное мороженое, как в детстве!",5
3081,"Мороженое Самокат эскимо, пломбир, с ванилью, ...",Очень вкусное мороженое,5
3082,"Мороженое Самокат вафельный стаканчик, шоколад...",Очень вкусное мороженое!,4
3083,"Мороженое Самокат вафельный стаканчик, шоколад...",Вкусное мороженое 👍,5


In [4]:
data.duplicated().sum()

298

Удалять дубликаты некорректно, люди могли написать что-то одинаковое

In [5]:
data = data.query('rating < 4')
data = data.reset_index(drop=True)

Уберем положительные отзывы, т.к. для поставленной задачи они не слишком важны. В них, конечно, может содержаться полезная информация для улучшения сервиса, но в первую очередь необходимо отработать негативные моменты

## Тематический анализ и частотный анализ слов

In [6]:
data['comment'] = data['comment'].str.replace('мороженое|мороженное', '')


C:\Users\stupa\AppData\Local\Temp\ipykernel_6336\2355543207.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['comment'] = data['comment'].str.replace('мороженое|мороженное', '')


Для более точного анализа слова уберем слово "мороженое", т.к. речь идет итак только о нем

In [7]:
reviews = data["comment"].tolist()

In [8]:
# Токенизация текста (разделение на отдельные слова)
def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True) if len(token) > 3]

# Создание словаря и корпуса
tokenized_reviews = [tokenize(review) for review in reviews]
dictionary = Dictionary(tokenized_reviews)
corpus = [dictionary.doc2bow(review) for review in tokenized_reviews]

# Обучение модели LDA
num_topics = 3  # Количество тем, которые хотим выделить
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

In [9]:
# Собираем все слова из отзывов в один список
all_words = [word for review in tokenized_reviews for word in review]

# Определяем 10 наиболее часто встречающихся слов
word_freq = Counter(all_words)
print(word_freq.most_common(10))


[('вкус', 94), ('очень', 74), ('было', 56), ('привезли', 49), ('шоколад', 34), ('глазурь', 33), ('мороженое', 33), ('растаявшее', 27), ('невкусное', 27), ('невкусно', 24)]


In [10]:
# Определение самой популярной темы для каждого отзыва
most_popular_topic_per_review = [max(review, key=lambda x: x[1])[0] for review in lda_model[corpus]]

In [11]:
reviews_with_most_popular_topic_0 = [i for i, topic in enumerate(most_popular_topic_per_review) if topic == 0]

for i in reviews_with_most_popular_topic_0:
    print(data.loc[i]['comment'])

Какое то безвкусное
Как будто порошок ела... Ну хотя бы холодным привезли😁
Ничего особенного  вкуса ириски я так и непоняла
Привезли упакованное откусанное 
непоняла прикола :(
В глазури хрустит сахар. У мороженного странный привкус.
Старая рецептура была вкуснее :(
Структура непонятная
Очень похоже на Коровку из Кореновки
неожидала таким жирным))
На любителей
неочень понравилось, но для своих денег норм
невкусно и дорого
Слишком завышенный ценник
На первый и после раз
Вместо вкуса мороженого - вкус сливочного масла. Пойдёт на бутерброды
В хвостике нет шоколадки :)
Вкусненько вроде, но опять повторюсь. Структура непонятная. Хотелось бы побольше воды, но чтобы вкус был более насыщенным. Но чувствуется что натуральное, хорошенькое. И мята с малиной здорово.
Ну ооооочень мало крошки((
Гипер сладкое
неочень, сильно чувствуется йогурт.
Больше похоже на за пирожное,  чем на  (стих получился)
Поддтаевшее и деформированное
Полурастаеный был. Аромат манго кстати даже вкус, но хз. незашел
рожок 

In [12]:
reviews_with_most_popular_topic_1 = [i for i, topic in enumerate(most_popular_topic_per_review) if topic == 1]

for i in reviews_with_most_popular_topic_1:
    print(data.loc[i]['comment'])

Какая то гадость купил детям выкинули, прогорклый вкус, кароче несоветую
Пластилиновый шоколад, который нетает во рту, пломбир без вкусный
Слишком жирно
Глазурь неожиданная на вкус и консистенцию, удивил состав глазури - хороший, вроде даже более полезный, чем обычная шоколадная глазурь, но непривычно
Слишком жирное
Это «» даже бомж есть небудет, нежели дети
невкусно(
Привезли растаявшее😒
Привезли растаявшее
Внутри вкусно. Но слой шоколада сверху как толстая плёнка, на вкус неочень
Очень невкусный шоколад ( Прилепает к зубам…
Вкус нормальный, но очень мало мороженного за такую цену
То ли горчит, то ли что, непонятно, но вкус непонравился :(
Состав
Испортился вкус. Возможно вы сменили поставщика.
Мороженое вкусное,, но попался кусок сливочного масла
Привезли почти растаявшим и такое ощущение , что в опой раз растаявшим )
очень жирное
Вафельный рожок мягкий, нехрустящий
Ещё бы вы его принесли нерастаявшим - цены бы небыло
невкусное вообще
Какое то кислое стало
Невкусно. Шоколад невкусный

In [13]:
reviews_with_most_popular_topic_2 = [i for i, topic in enumerate(most_popular_topic_per_review) if topic == 2]

for i in reviews_with_most_popular_topic_2:
    print(data.loc[i]['comment'])

На любителя
Растаяло
Раньше было вкуснее :( сейчас слишком горькое
Раньше было вкуснее
Дорогое 
2 мороженого были заново переморожены, лёд внутри и подтеки
Очень невкусная и странная «шоколадная глазурь»
Кислое, невкусное, как будто заморрзили невкусный несладкий йогурт, непонравилось никому из семьи, даже ребенку. Раньше было вкусное клубничное тоже типа с йогуртом, это совсем на него непохоже
Привезли помятое. Такое ощущение , что растаяло, потом опять замёрзло. Ну и, соответственно лёд на зубах хрустит. Неприятно
Нет выраженного вкуса Брюле.
Слишком много сахара для пломбира
Странное послевкусие, непохоже на пломбир
Сильно подтаявшее
Мыльное послевкусие
Средне
невкусная глазурь сверху мороженного. Она неотламывается при откусывании (как у др.производителец мороженого), а прилипает к зубам как пластилин.
на любителя. как замороженный лед фруктовый
невкусная глазурь(
Такое редкостное Г!!!
невкусная глазурь, мягкая
Вообще невкусное, как сметана или сливки, несладкое, съела пару ложек и

In [14]:
# Получение списка тем и наиболее вероятных слов для каждой темы
topics_words = lda_model.show_topics(num_topics=num_topics, num_words=10)

# Вывод наиболее популярных слов для каждой темы
for topic_id, words in topics_words:
    print(f"Тема {topic_id}: {words}")


Тема 0: 0.019*"привезли" + 0.019*"вкус" + 0.013*"горчит" + 0.009*"очень" + 0.008*"мало" + 0.007*"сильно" + 0.007*"вкуса" + 0.007*"сладкое" + 0.007*"слишком" + 0.006*"чувствуется"
Тема 1: 0.032*"вкус" + 0.027*"очень" + 0.015*"шоколад" + 0.013*"привезли" + 0.010*"невкусно" + 0.010*"вкусное" + 0.010*"растаявшее" + 0.009*"мороженое" + 0.009*"невкусныи" + 0.008*"глазурь"
Тема 2: 0.036*"было" + 0.014*"растаяло" + 0.012*"глазурь" + 0.011*"невкусное" + 0.011*"мороженое" + 0.010*"вкуснее" + 0.008*"очень" + 0.008*"будто" + 0.008*"раньше" + 0.008*"невкусная"


Исходя из визуального анализа каждой темы и частотного анализа слов для каждой из них можно выделить 3 основных проблемных темы:   
Проблемы со вкусом, проблемы с доставкой, и "раньше было лучше"

In [15]:
# последовательность и наполнение этих тем меняются при каждом перезапуске, но это не мешает составить именно такой выбор тем

Посмотрим на проблемы в определенных темах

In [20]:
topics_keywords = {
    "taste": ['горьк', 'горч', 'сладк', 'невкус', 'шокол', 'глазу', 'ненатурал', 'ужас'],
    "delivery": ['таявш', 'раста',  'курьер', 'упаков', 'привез'],
    "nostalgia": ['раньш', 'было', 'лучше', 'вкуснее', 'партия']
}

for topic, keywords in topics_keywords.items():
    print(f"Тема: {topic}")
    total_count = 0
    for keyword in keywords:
        count = data["comment"].str.contains(keyword, case=False).sum()
        total_count += count
        print(f"{keyword}: {count}")
    print(f"Итог: {total_count}\n")

Тема: taste
горьк: 24
горч: 19
сладк: 27
невкус: 77
шокол: 61
глазу: 35
ненатурал: 3
ужас: 33
Итог: 279

Тема: delivery
таявш: 52
раста: 78
курьер: 6
упаков: 17
привез: 50
Итог: 203

Тема: nostalgia
раньш: 15
было: 52
лучше: 7
вкуснее: 15
партия: 1
Итог: 90



## Наиболее проблемные товары

In [27]:
data.groupby('name_samokat', as_index = False).agg({'rating':['mean', 'count']}).sort_values(by=('rating', 'mean'))

name_samokat    rating      
                                                           mean count
9   Мороженое Самокат вафельный стаканчик, пломбир...  1.714286    42
22  Мороженое Самокат эскимо, пломбир, с ароматом ...  1.750000     4
17  Мороженое Самокат с йогуртом, с клубникой, 10%...  1.800000   100
14              Мороженое Самокат пломбир, 15%, 270 г  1.800000    10
8   Мороженое Самокат в вафельном стаканчике, плом...  1.810811    37
16  Мороженое Самокат пломбир, шоколадный, 15%, в ...  1.812500    16
18  Мороженое Самокат стаканчик, сливочное, со вку...  1.875000     8
12  Мороженое Самокат пломбир классический, 15%, 2...  1.882353    17
21  Мороженое Самокат эскимо, пломбир с ароматом в...  1.895522    67
7   Мороженое Самокат в вафельном стаканчике, вани...  1.896552    29
10  Мороженое Самокат вафельный стаканчик, пломбир...  1.961538    26
6   Мороженое Самокат в вафельном рожке, пломбир в...  2.000000     2
20  Мороженое Самокат эскимо, двухслойный пломбир,...  2.097561    41
23  Мороженое Самокат эскимо, пломбир, с ванилью, ...  2.103448    29
5   Мороженое Самокат в вафельном рожке, двухслойн...  2.162162    37
13   Мороженое Самокат пломбир шоколадный, 13%, 270 г  2.200000     5
19  Мороженое Самокат эскимо, Бостон-Брауни класси...  2.200000    20
3   Десерт растительный Самокат маракуйя с манго и...  2.218750    32
11  Мороженое Самокат вафельный стаканчик, шоколад...  2.222222    27
1   Десерт растительный Самокат гранат и вишня, за...  2.222222    18
15  Мороженое Самокат пломбир, с шоколадной крошко...  2.285714    21
4   Мороженое Самокат Трубочка, двухслойное, в гла...  2.319149    47
2   Десерт растительный Самокат малина с зеленью м...  2.500000    18
0   Десерт растительный Самокат банан и клубника, ...  2.500000     4

## Вывод

1. Наибольшее недовольство пользователей вызывают вкусовые характеристики товара. Невкусные шоколад/глазурь, горько, сладко и т.д.
2. Следующее недовольство вызвано доставкой. Привезли растаявшим или были проблемы с упаковкой
3. Еще одна причина недовольства "раньше было лучше". Это может быть вызвано субъективным восприятием клиентов или же изменением технологий производства

## Предложения и гипотезы по улучшению опыта пользователей:


1. По вкусовым характеристикам: выбрать наиболее проблемные товары (в совокупности по количеству заказов и средней оценке), детально изучить отзывы, попробовать изменить технологию производства и вкус товара, испытать его на различных дегустационных группах. После достижения устраиваемого результата возобновить поставки и следить за динамикой оценок и отзывами о вкусе  
2. По проблемам с доставкой: установить регламенты для курьеров и ПВЗ (не собирать мороженое в заказ до прибытия курьера, хранить в термосумке и тд); установить время доставки с мороженым, чтобы оно не успело растаять; для отслеживания изменений необходимо привязать к данным время доставки и время сборки заказа, смотреть за корелляцией отзывов и временем.
3. Решение проблемы "раньше было лучше" аналогичное первой проблемы. Но от ностальгических чувств никогда не получится избавить польностью, это слишком субъективно :)